In [1]:
import sys

import numpy as np

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


# Setup the model area and river geometry

In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)

river = LineString([
  (13.9226, 50.9661),
  (13.9250, 50.9657),
])


In [3]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.add_child(folium.GeoJson(river.as_geojson()))

m.fit_bounds(m.get_bounds())
m

## Calculate and Render Grid Perimeter and River Cells 

In [4]:
from morpheus.project.types.discretization.spatial import ActiveCells

grid_geometry = grid.to_geojson()

cells = ActiveCells.from_linestring(river, grid)
cells_geometries = cells.to_geojson(grid)

m.add_child(folium.GeoJson(grid_geometry.as_geojson()))
m.add_child(folium.GeoJson(cells_geometries.as_geojson()))
m.fit_bounds(m.get_bounds())
m

## Calculate distance on the line for each cell
This consists of two steps:
1. Find the nearest point on the line from each cell center
2. Calculate the distance from the start of the line to the nearest point

In [5]:
from shapely import LineString as ShapelyLineString, Polygon as ShapelyPolygon

# River LineString
line = ShapelyLineString(river.coordinates)

# Cell Centers
centers = [ShapelyPolygon(cell.coordinates[0]).centroid for cell in cells.to_geojson(grid).geometries]

# Normalized distance on the line from the start to the nearest point on the line for each cell
distances_on_line_to_nearest_points = [line.project(center, normalized=True) for center in centers]
np.array(distances_on_line_to_nearest_points)

array([0.03481602, 0.15750677, 0.28019751, 0.41349572, 0.53618646,
       0.65887721, 0.79217545, 0.91486619, 1.        ])

In [6]:
# for the linear interpolation we need to set our known points and values (xx, vv)
xx = [0, 0.5, 1]
vv = [10, 0, 20]

# And we can set our new points (xx_new) and calculate the new values (vv_new)
xx_new = distances_on_line_to_nearest_points
vv_new = np.interp(xx_new, xx, vv)
vv_new

array([ 9.30367954,  6.84986466,  4.39604979,  1.73008565,  1.44745845,
        6.35508821, 11.68701791, 16.59464766, 20.        ])